<span type="title">Tag Files and Custom Tag</span> | <span type="update">2018-09-25</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章首先介绍了标记文件，这是一种用标记来进行页面包含的标准。其次介绍了自定义标记和标记处理器，有了自定标记，我们就可以进行更为方便的、更加解耦的开发。虽然EL+JSTL能提供大多数的View层的逻辑和计算操作，但是当我们希望新建一个Java对象，在数个Java对象之间执行一系列动作，最后获取输出的时候，自定标记就可以派上用场了。区别于EL的静态函数，自定标记可以传递属性和体，并且可以动态调用变量格式化输出，其功能更为强大。</p><p class="card-text">在这篇文章中的第一部分，介绍了使用标记文件简化包含以及参数传递。第二部分主要介绍了简单标记处理器以及自定标记，以及父子标记的互相调用和动态属性实现。自定标记可以弥补EL+JSTL对于显示页面的逻辑上的不足，但是，也不能滥用，对于模型应该做的事情，应该交给模型来实现逻辑。此外，近年的前后端分离也导致了标准动作+EL+JSTL+自定标签这种解决方案用的越来越少。</p></span>

# 标记文件

标记文件指的是通过使用标签的方式来进行文件包含的一种标准。这是为了解决：在一个JSP页面需要包含大量其余页面，但是 jsp:include 和 include 指令以及 c:import 使用混乱的问题。在 JSP 2.0 标准之前，一个 Web 程序往往带有很多包含的页面片段，难以管理和阅读。使用标记文件可以通过一个标记前缀来访问指定目录下的任意页面片段，阅读更加容易。

```jsp
<%@ taglib prefix="t" tagdir="/WEB-INF/tags" %>
<t:header_img />
<t:menu title="Marvin">This is a description.</t:menu>
```

作为标记，肯定是要使用 taglib 指令，区别在于，我们没有像标准动作或者JSTL那样指定 uri，而是指定了一个文件夹 tagdir，这个文件夹中所有的以 .tag 结尾的文件都可以被直接调用。比如 `<t:header_img>` 可以调用 /WEB-INF/tags/header_img.tag 处的片段。这个 tag 本质上是一个 jsp。

标记文件可能在以下文件夹进行搜索：

- WEB-INF/tags 及其子文件夹，比如 WEB-INF/tags/header_tags/
- WEB-INF/lib 下任意 JAR 的 META-INF/tags 及其子文件夹

需要注意，如果在一个 JAR 下，则需要指定 TLD。

现在的参数传递也更简单了，直接使用标签的属性即可。而对于属性的要求和限制也不放在 TLD 文件中，而是放在这些被包含的页面片段本身。

比如 menu.tag 的内容如下：

```jsp
<%@ tag body-content="scriptless" dynamic-attributes="args" %>
<%@ attribute name="title" required="true" description="name of menu" rtexprvalue="true" %>
<h1>Hello ${title}</h1>
<p><jsp:doBody/></p>
```

注意这里又一些特殊指令，比如 tag，attribute，jsp:doBody等，只能用于 .tag 文件。

其中 tag 指令用于设置是否有体，是否接受动态参数等，默认为 scriptless。动态参数传递的是一个映射，这个映射被保存为本地变量 args。

attribute 指令用来设定属性，包括名称、描述、是否强制、是否允许计算。对于属性，其都被设置为 pageScope 的属性，以便于 EL 表达式访问。

对于体中的值，用 `<jsp:doBody>` 来获取。

tag 文件本身就是一个 jsp，因此可以使用 jsp 文件的隐式对象。注意，ServletContext 在 JSP 中是 JspContext。

# 定制标记

标记文件使用 taglib 解决了片段包含碎片化的问题。定制标记的目的和它关系不大，主要是为了满足 EL + JSTL 对于输出视图页面的 HTML 和 Java代码之间转换的需求不能很好满足的问题。其中最关键的就是，JSTL 和 EL，包括标准动作，都不能新建java对象（虽然可以传递值、遍历、判断和输出），提供的EL函数支持只支持静态方法。

我们的解决方法是，像使用 taglib 解决片段包含的问题一样，使用自定义的标签前缀以及标签名称来解决使用Java代码输出HTML的问题。这和 taglib 中的 function 用来建立 Java 和 HTML 的桥梁的作用一样，不过定制标记更主要侧重于 Java 代码，适合复杂逻辑，能够自定义大量变量，你可以动态的格式化输出结果，而 function 只能是静态方法，适合简单的 Java 对象调用和简单的逻辑。

实现定制标记可以使用传统的或者简单两种路子，对于简单定制标记模型，现在更为常用，其核心是实现一个简单标记处理器。

## 简单标记处理器

简单标记处理器需要扩展SimpleTagSupport类，覆盖doTag方法，为标记创建一个独立的，在指定路径下的TLD，这个TLD应该指定此Java类的位置、名称以及是否有体，以及是否允许属性，是否有动态属性，有什么属性，是否必须，是否动态生成等。之后在JSP声明此 taglib，使用标签进行调用。

SimpleTagSupport 实现了 SimpleTag 接口，而后者则实现了 JspTag 接口，JspTag 建立起了传统和简单模型的桥梁。SimpleTag 提供了对于嵌套、参数、体的方法。而在 SimpleTagSupport 中，添加了 doTag 方法，这个方法中，你可以进行自己的操作。其生命周期如下：

首先是加载和初始化类，之后调用setJspContext方法，将PageContext传递到这个处理器中，方便寻找属性和输出流。之后，如果是嵌套的，调用setParent(Tag)来进入嵌套处理器进行设置（类似于继承的super递归初始化）。之后将所有的属性都使用setXXX()来传递到处理器，这部分需要你自己实现。最后，如果有体（DD不允许有体，体为空，没有写体这三者之外），那么调用setJspBody(JspFragment)来设置体，最后是我们的doTag方法，在这里进行逻辑实现，最后结束生命周期。需要注意，对于每次请求，不会重用这个处理器，会重新实例化一个新的标记处理器。

下面简要介绍一个例子：

注意，这里的 number 如果不存在于 pageScope 的话（也就是标记使用了表达式），在 doTag 中需要进行设置和处理。

```jsp
<%@ taglib prefix="m" uri="http://java.mazhangjing.com/tag/simple" %>
<m:upper attr="hi">This is something good.${number}</m:upper>
```

```xml
<uri>http://java.mazhangjing.com/tag/simple</uri>
<tag>
    <name>upper</name>
    <tag-class>com.mazhangjing.model.Upper</tag-class>
    <body-content>scriptless</body-content>
    <attribute>
        <name>attr</name>
        <rtexprvalue>false</rtexprvalue>
    </attribute>
</tag>
```

```java
public class Upper extends SimpleTagSupport {
    private String attr;
    @Override
    public void doTag() throws JspException, IOException {
    
        //当没有体时，我们直接调用输出
        getJspContext().getOut().print(" ");

        //当体中没有EL表达式时，我们希望对于体进行操作：
        StringWriter writer = new StringWriter();
        getJspBody().invoke(writer);
        getJspContext().getOut().print(writer.toString().toUpperCase());
        
        //当体中有EL表达式时，我们一般希望将表达式转换成为字符串，使用属性设置和invoke打印来做到这一点。
        for (int i = 0; i < 5; i++) {
            getJspContext().setAttribute("number",i*1000);
            getJspBody().invoke(null);
        }

        //如果希望页面余下部分都停止解析，而同时不影响页面以上的部分，就不能抛出JspException，而是要使用：
        throw new SkipPageException();
    }

    public String getAttr() {return attr;}

    public void setAttr(String attr) {this.attr = attr;}
}
```

doTag 常用的方法有两个 getJspContext 提供了设置和获取 Attritube、out 输出流的途径。 getJspBody 提供了获取体和将体写入到输出流的途径。注意这个 JspBody，是一个 JspFragment 对象，它不包含任何脚本，但是可以包含 EL，模板HTML，以及标准和定制动作。我们甚至可以获得这个 JspFragment 交给别的类进行处理。因为它提供了一个 JspContext 上下文，用来指示内部变量来源和输出流。

在 doTag 中进行的处理主要分为这几种情况：

- 当没有体时，我们可以直接调用 context 得到的 out 来输出信息。


- 当有体，但是体中没有EL时，我们可以截获 body 中的内容（通过invoke到一个String中），然后进行处理后，打印到 context 的 out 输出流中。


- 当有体，但是体中包含EL变量时，我们常常先调用 context.setAttritube 来设置属性，后调用 body.invoke(null) 来格式化输出变量的结果。我们甚至可以使用循环重复输出一个iterable的值。这里的invoke(null)指的就是获取 JspFragment，然后输出到 out 中。因为格式化的时候，我们需要做的是定义好变量，不需要对输出流进行其他控制，把具体的输出结果交给JSP去动态生成。

除了 doTag，我们需要为每个属性添加一个 Bean 待遇的 set 和 get 方法。方便容器在处理标签时调用并且传递属性到标签处理器。如果想要获取request等对象，将jspcontext转型为pagecontext，然后获取即可。如果希望停止此标签以下的所有输出，使用 SkipPageException，这会保留此标签之前的输出，但是停止此标签所在页面以下的解析。注意，如果这个页面被包含，其不会影响包含页面（父页面）的解析。

## 动态属性的实现

像 args 动态参数一样，如果我们要传递大量属性，那么使用 getXXX 和 setXXX 很麻烦。因此需要使用动态属性，需要让标记处理器实现 DynamicAttributes 接口，设置一个 hashMap 用来存储数据，提供一个 set的方法即可。

此外，在 TLD 中，需要声明：
```xml
<tag>
    <name>upper</name>
    <tag-class>com.mazhangjing.model.Upper</tag-class>
    <body-content>scriptless</body-content>
    <attribute>
        <name>attr</name>
        <rtexprvalue>false</rtexprvalue>
    </attribute>
    <dynamic-attributes>true</dynamic-attributes>
</tag>
```

```java
public class Upper extends SimpleTagSupport implements DynamicAttributes {
    private Map<String,Object> args = new HashMap<>();
    @Override
    public void setDynamicAttribute(String s, String s1, Object o) {
        args.put(s1,o);
    }
}
//像这样使用：
for (String id : args.keySet()) {
    getJspContext().getOut().print(id + " ==>" + args.get(id).toString());
}
```

对于标记文件的动态属性，也是类似，需要声明 dynamic-attributes，然后直接使用（区别是不需要自己手动构建hashMap）。menu.tag 举例如下：

```jstl
<%@ taglib prefix="c" uri="http://java.sun.com/jsp/jstl/core" %>
<%@ tag body-content="scriptless" dynamic-attributes="args" %>
<c:forEach items='${args}' var="item" varStatus="status">
    <p>${status.count} - ${item}</p>
</c:forEach>
```

## 标记的嵌套

标记可以互相嵌套，那么自然有从父或者子标记中取数据的需求。对于子标记而言，使用getParent方法即可获取父标记的引用，

```jsp
<m:upper attr="father"><m:upper attr="son"/></m:upper>
```
```java
System.out.print(
    (Upper) getParent()).getJspContext().getAttribute("attr").toString());
```

而父标记要调用子标记，则需要将子标记设置为父标记的一个属性，然后进行设置。不管谁调用谁，都需要注意 null 值的问题。

此外，子标记寻找父标记可以使用 `findAncestorWithClass(this,FatherClass.class)`，找到即停止。不过一般很少用。